In [ ]:
# 1. Nạp dữ liệu (Load Data)
# Đọc tất cả file âm thanh từ dataset Kaggle (thường dạng .wav, .mp3).
# Gắn nhãn cảm xúc từ tên file hoặc metadata (ví dụ: 03-01-05-01-01-01-01.wav → Emotion: Angry).
# Bắt đầu : Làm trên vicode: 
# Nạp dữ liệu 
# Phân loại dữ liệu

# 2. Chuẩn hóa định dạng (Standardization)
# Convert về mono (1 kênh) nếu file gốc là stereo.
# Chọn sample rate chuẩn (thường 16kHz hoặc 22.05kHz).
# Lý do: giúp thống nhất tần số lấy mẫu, tránh dữ liệu không đồng nhất.
# Dùng librosa.load(path, sr=16000, mono=True) để thực hiện.

# 3. Làm sạch tín hiệu (Noise Reduction)
# Cắt đoạn im lặng ở đầu/cuối (silence trimming).
# Loại bỏ nhiễu:
# Filter (high-pass, low-pass).
# Hoặc thêm bước spectral gating nếu cần.

# 4. Chuẩn hóa thời lượng (Duration Normalization)
# Các file thường có độ dài khác nhau → cần đồng nhất.
# Hai cách thường làm:
# Padding: nếu file ngắn hơn chuẩn (ví dụ 3s) → thêm 0 cho đủ.
# Truncate: nếu file dài hơn → cắt bớt.

# Mục tiêu: tất cả file có cùng số frame để dễ batch training.


# 5. Chuẩn hóa biên độ (Amplitude Normalization)
# Scale tín hiệu về cùng mức cường độ.
# Ví dụ: chuẩn hóa dữ liệu về [-1, 1].
# Tránh việc file có âm lượng quá to/nhỏ làm sai lệch đặc trưng.

# 6. Data Augmentation (Tăng dữ liệu ảo)
# (không bắt buộc nhưng rất thường dùng)
# Thêm biến thể để mô hình học tốt hơn:
# Time-stretching (nói nhanh/chậm).
# Pitch-shifting (tăng/giảm cao độ).
# Add background noise (âm quán cà phê, văn phòng).
# Random crop (cắt đoạn bất kỳ trong file dài).



# 7. Lưu trữ & Quản lý (Save Processed Data)
# Lưu các file đã tiền xử lý để không phải xử lý lại nhiều lần.cd
# Có thể lưu dưới dạng:
# MFCC arrays (numpy .npy).
# Spectrogram images (PNG/JPG để huấn luyện CNN).
# Hoặc giữ file .wav đã chuẩn hóa.


# 📊 Sơ đồ tóm tắt workflow
# Nạp dữ liệu → Chuẩn hóa định dạng → Làm sạch tín hiệu → Chuẩn hóa thời lượng → Chuẩn hóa biên độ → (Tăng dữ liệu ảo) → Lưu trữ đặc trưng


In [ ]:
# This code includes all the necessary methods for audio preprocessing:

# Loading and standardizing audio files
# Noise reduction
# Duration normalization
# Amplitude normalization
# Data augmentation
# File processing
# Directory processing

In [ ]:
# Cấu trúc thư mục đề xuất:
# code_speech/
# ├── data/
# │   ├── angry/
# │   ├── happy/
# │   ├── neutral/
# │   ├── sad/
# │   ├── surprise/
# │   ├── fear/
# │   └── disgust/
# └── backend/
#     └── preprocessing/
#         └── roadmap.ipynb

In [ ]:
# pip install numpy librosa soundfile

In [ ]:
import os
import numpy as np
import librosa
import soundfile as sf
from typing import Tuple, List
from pathlib import Path
from tqdm import tqdm
from scipy.signal import butter, filtfilt  # Thêm thư viện cho lọc nhiễu

class AudioPreprocessor:
    def __init__(self, 
                 target_sr: int = 16000,
                 target_duration: float = 3.0,
                 target_dbfs: float = -20):
        self.target_sr = target_sr
        self.target_duration = target_duration
        self.target_dbfs = target_dbfs
        
    def load_audio(self, file_path: str) -> Tuple[np.ndarray, int]:
        """Load and standardize audio format"""
        # Load audio with target sample rate and convert to mono
        audio, sr = librosa.load(file_path, sr=self.target_sr, mono=True)
        return audio, sr
    
    def butter_highpass(self, cutoff, fs, order=5):
        """Thiết kế bộ lọc highpass"""
        nyq = 0.5 * fs
        normal_cutoff = cutoff / nyq
        b, a = butter(order, normal_cutoff, btype='high', analog=False)
        return b, a

    def reduce_noise(self, audio: np.ndarray) -> np.ndarray:
        """Loại bỏ nhiễu và khoảng lặng"""
        # Cắt khoảng lặng
        audio_trimmed, _ = librosa.effects.trim(audio, top_db=20)
        
        # Lọc tần số thấp sử dụng bộ lọc Butterworth
        b, a = self.butter_highpass(cutoff=20, fs=self.target_sr)
        audio_filtered = filtfilt(b, a, audio_trimmed)
        
        return audio_filtered
    
    def normalize_duration(self, audio: np.ndarray) -> np.ndarray:
        """chuẩn hóa độ dài"""
        target_length = int(self.target_sr * self.target_duration)
        
        if len(audio) > target_length:
            # Cắt bớt
            audio = audio[:target_length]
        else:
            # Thêm 0
            padding = target_length - len(audio)
            audio = np.pad(audio, (0, padding), mode='constant')
            
        return audio
    
    def normalize_amplitude(self, audio: np.ndarray) -> np.ndarray:
        """Chuẩn hóa biên độ về [-1,1]"""
        audio = audio / np.max(np.abs(audio))
        return audio
    
    def augment_audio(self, audio: np.ndarray) -> List[np.ndarray]:
        """Tạo dữ liệu tăng cường"""
        augmented = []
        
        # Thay đổi tốc độ
        audio_stretch = librosa.effects.time_stretch(audio, rate=0.9)
        augmented.append(audio_stretch)
        
        # Thay đổi cao độ
        audio_pitch = librosa.effects.pitch_shift(
            audio,
            sr=self.target_sr,
            n_steps=2)
        augmented.append(audio_pitch)
        
        return augmented
    
    def process_file(self, file_path: str, output_dir: str, augment: bool = False):
        """Xử lý một file âm thanh"""
        # Lấy tên file 
        filename = os.path.basename(file_path)
        
        # Nạp âm thanh
        audio, sr = self.load_audio(file_path)
        
        # Áp dụng các bước xử lý 
        audio = self.reduce_noise(audio)
        audio = self.normalize_duration(audio)
        audio = self.normalize_amplitude(audio)
        
        # Lưu file đã xử lý
        output_path = os.path.join(output_dir, f"processed_{filename}")
        sf.write(output_path, audio, sr)
        
        # Tạo dữ liệu tăng cường nếu cần 
        if augment:
            augmented = self.augment_audio(audio)
            for i, aug_audio in enumerate(augmented):
                aug_path = os.path.join(output_dir, f"aug{i+1}_{filename}")
                sf.write(aug_path, aug_audio, sr)

    def process_directory(self, input_base_dir: str, output_base_dir: str, augment: bool = True):
        """Process all audio files in the emotion directories"""
        # Create output base directory
        os.makedirs(output_base_dir, exist_ok=True)
        
        # Get list of emotion folders from filesystem
        emotion_dirs = ['angry', 'happy', 'neutral', 'sad', 'surprise', 'fear', 'disgust']
        
        #count total files for progress tracking 
        total_files = 0
        for emotion in emotion_dirs:
            input_dir = os.path.join(input_base_dir, emotion)
            if os.path.exists(input_dir):
                files = [f for f in os.listdir(input_dir) if f.endswith(('.wav', '.mp3'))]
                total_files += len(files)
            
        print(f"Total files to process: {total_files}")

        #Initialize processed bar
        pbar = tqdm(total=total_files, desc="Processing files")
        for emotion in emotion_dirs:
            input_dir = os.path.join(input_base_dir, emotion)
            output_dir = os.path.join(output_base_dir, emotion)
            
            # Skip if emotion folder doesn't exist
            if not os.path.exists(input_dir):
                print(f"Skipping {emotion} - directory not found")
                continue
                
            # Create output directory for this emotion
            os.makedirs(output_dir, exist_ok=True)
            
            # Process each file in the emotion directory
            files = [f for f in os.listdir(input_dir) if f.endswith(('.wav', '.mp3'))]
            
            print(f"\nProcessing {emotion} files: {len(files)} files found")
            for file in files:
                try:
                    input_path = os.path.join(input_dir, file)
                    self.process_file(input_path, output_dir, augment=augment)
                    pbar.update(1)
                    print(f"✓ Processed: {file}")
                except Exception as e:
                    print(f"✗ Error processing {file}: {str(e)}")
                    continue
        pbar.close()

if __name__ == "__main__":
    # Install required package if not present 

    try:
        from tqdm import tqdm
    except ImportError:
        import pip
        pip.main(['install', 'tqdm'])
        from tqdm import tqdm

    # Khởi tạo đường dẫn 
    project_root = "D:\\filebailam\\CD3_Giongnoi\\code_speech"
    
    # Setup input/output directories
    input_dir = os.path.join(project_root, "data", "Emotion")
    output_dir = os.path.join(project_root, "data", "processed")
    
    print(f"Input directory: {input_dir}")
    print(f"Output directory: {output_dir}")
    

    try:
        # Initialize preprocessor
        preprocessor = AudioPreprocessor(
            target_sr=16000,      # 16kHz sample rate
            target_duration=3.0,   # 3 seconds duration
            target_dbfs=-20       # -20 dBFS target level
        )
        
        # Process all files
        preprocessor.process_directory(input_dir, output_dir, augment=True)
        print("\nProcessing complete!")
    except KeyboardInterrupt:
        print("\nProcessing interrupted by user.")
    except Exception as e:
        print(f"\nAn error occurred: {str(e)}")

Input directory: D:\filebailam\CD3_Giongnoi\code_speech\data\Emotion
Output directory: D:\filebailam\CD3_Giongnoi\code_speech\data\processed
Total files to process: 19777


Processing files:   0%|          | 0/19777 [00:00<?, ?it/s]


Processing angry files: 3495 files found


Processing files:   0%|          | 4/19777 [00:07<8:19:56,  1.52s/it] 

✓ Processed: 0001_000352.wav
✓ Processed: 0001_000353.wav
✓ Processed: 0001_000354.wav
✓ Processed: 0001_000355.wav
✓ Processed: 0001_000358.wav


...

Processing files:  32%|███▏      | 6331/19777 [05:27<11:29, 19.51it/s]

✓ Processed: 0014_000995.wav
✓ Processed: 0014_000996.wav
✓ Processed: 0014_000997.wav
✓ Processed: 0014_001000.wav
✓ Processed: 0014_001001.wav


Processing files:  32%|███▏      | 6336/19777 [05:27<11:41, 19.16it/s]

✓ Processed: 0014_001002.wav
✓ Processed: 0014_001004.wav
✓ Processed: 0014_001005.wav
✓ Processed: 0014_001006.wav

Processing interrupted by user.


Processing files:  32%|███▏      | 6337/19777 [05:40<11:41, 19.16it/s]

In [ ]:
## thử nghiệm quy trình 